In [ ]:
# Imports
import numpy as np
from scipy import stats
from scipy.special import comb

### Getting Started
This notebook provides helpful formulas for computing optimal parameters for the construction of B-field (described further [here](https://github.com/onecodex/rust-bfield)). It includes a few sections:
* **Quick Calculator**: Change a few input variables to determine optimal B-field construction parameters
* **Space Efficiency vs. Error Rate**: Visualize B-field space efficiency vs. error rate for B-fields supporting several different maximum numbers of values ($\theta$).

In [ ]:
def calculate_nu_and_kappa(max_value, max_nu=64):
    """Find ν and κ with a constraint of a `max_nu` value, minimizing κ.
    """
    nu = 2
    kappa = 1
    while kappa < nu:
        for nu in range(1, max_nu + 1):
            if comb(nu, kappa) >= max_value:
                return nu, kappa
        kappa += 1
    raise Exception(f"No value of ν choose κ has a value over {max_value}. Consider raising the `max_nu` parameter.")
    
    
def calculate_fp_rate(m_over_n, n_hashes):
    return np.power(1 - np.power(np.e, -n_hashes * 1 / m_over_n), n_hashes)
    
    
def calculate_m_over_n_and_hashes_from_per_bit_fp(max_per_bit_fp, max_hashes=12):
    """Find an optimal number of hashes, k, and m/n (bits per element), minimizing m/n
    
    See https://pages.cs.wisc.edu/~cao/papers/summary-cache/node8.html for helpful detail.
    """
    m_over_n = 2
    fp_rate = np.inf
    while fp_rate >= max_per_bit_fp:
        for n_hashes in range(1, max_hashes + 1):
            fp_rate = calculate_fp_rate(m_over_n, n_hashes)
            if fp_rate < max_fp_rate:
                return m_over_n, n_hashes
        m_over_n += 1
    raise Exception(f"No m/n found for max false positive rate of {max_fp_rate}. Consider increasing `max_hashes` parameter.")

    
def calculate_m_over_n_and_hashes_from_alpha(max_alpha, max_hashes=12):
    """Find an optimal number of hashes, k, and m/n (bits per element), minimizing m/n    
    """
    m_over_n = 2
    alpha = np.inf
    while alpha >= max_alpha:
        for n_hashes in range(1, max_hashes + 1):
            fp_rate = calculate_fp_rate(m_over_n, n_hashes)

            # We skip anything where we're in the lefthand side of the CDF
            if stats.binom.cdf(kappa, nu, fp_rate) < 0.5:
                continue
                
            alpha = stats.binom.cdf(kappa, nu, fp_rate) - stats.binom.cdf(kappa - 1, nu, fp_rate)
            if alpha < max_alpha:
                return m_over_n, n_hashes, alpha
        m_over_n += 1
    raise Exception(f"No m/n found for max false positive rate of {max_fp_rate}. Consider increasing `max_hashes` parameter.")
        

### Quick Calculator
Set the following configuration options and then run the cell to compute the required B-field creation parameters:
* `MAX_VALUE`: The maximum value $y$ you'd like to store (alternatively $\theta$). Note the `rust-bfield` implementation only supports `u32` integers for values and you should strongly consider remapping values to a complete range of natural numbers $1...\theta$.
* `MAX_FALSE_POSITIVE_RATE`: The maximum false positive rate $(\alpha)$ you'd like to allow in your B-field. Recommended values for many applications are 0.01 or below.
* `MAX_INDETERMINACY_RATE`: The maximum indeterminacy rate $(\beta)$ you'd like to allow in your B-field. Recommend a value of 0.

In [ ]:
MAX_VALUE = 1e6
MAX_FALSE_POSITIVE_RATE = 0.001
MAX_INDETERMINACY_RATE = 0
N_ELEMENTS = 1e9

# Recommended standard values
MAX_SCALEDOWN = 0.001

# First we find suitable values of nu and kappa
nu, kappa = calculate_nu_and_kappa(MAX_VALUE)

# Then we compute the bits per element required for the desired false positive rate on a per bit basis
m_over_n, n_hashes, alpha = calculate_m_over_n_and_hashes_from_alpha(MAX_FALSE_POSITIVE_RATE)

p = calculate_fp_rate(m_over_n, n_hashes)
bits_per_element = m_over_n * kappa

# Next, we compute the implied indeterminacy error rate and the required number and size of secondary arrays
uncorrected_beta = stats.binom.cdf(1, nu - kappa, p) - stats.binom.cdf(0, nu - kappa, p)  # this is also the scaledown factor
n_secondaries = 0
calculated_indeterminacy_rate = np.inf

#
secondary_array_size = N_ELEMENTS
expected_indeterminate_results = int(N_ELEMENTS * uncorrected_beta)
array_sizes = []
debug = False
while calculated_indeterminacy_rate > MAX_INDETERMINACY_RATE:
    # Stop if the expected number of indeterminate results is < 0.5    
    array_sizes.append(secondary_array_size * bits_per_element)
    if expected_indeterminate_results < 0.5:
        break

    # Scale the secondary array down by the uncorrected 𝛽
    n_secondaries += 1    
    secondary_array_size = int(secondary_array_size * uncorrected_beta)
    
    # But never make an array smaller than N_ELEMENTS * MAX_SCALEDOWN
    if secondary_array_size < N_ELEMENTS * MAX_SCALEDOWN:
        secondary_array_size = int(N_ELEMENTS * MAX_SCALEDOWN)

    if debug:
        print(f"The #{n_secondaries} secondary array will be {secondary_array_size:,} elements ({int(expected_indeterminate_results):,} expected elements)")
        
    # Now calculate the expected number of indeterminate results flowing *out* of the nth secondary array
    secondary_array_size_bits = secondary_array_size * bits_per_element
    corrected_m_over_n = (secondary_array_size / expected_indeterminate_results) * m_over_n
    corrected_p = calculate_fp_rate(corrected_m_over_n, n_hashes)
    
    # Heuristic: But don't allow p to be set to 0, always use at least 10-e7 (1 in 1M)
    corrected_p = max(10e-7, corrected_p)
    corrected_beta = stats.binom.cdf(1, nu - kappa, corrected_p) - stats.binom.cdf(0, nu - kappa, corrected_p)
    expected_indeterminate_results = expected_indeterminate_results * corrected_beta
    
    if debug:
        print(f"Expect {int(expected_indeterminate_results):,} indeterminate results in next array ({corrected_m_over_n}, corrected p {corrected_p:.10f}), corrected beta {corrected_beta:.4f}")

In [ ]:
print(f"""
Input configuration requirements are:

`MAX_VALUE` (𝜃) = {int(MAX_VALUE):,}
`MAX_FALSE_POSITIVE_RATE` (𝛼) = {MAX_FALSE_POSITIVE_RATE}
`MAX_INDETERMINACY_RATE` (corrected 𝛽) = {MAX_INDETERMINACY_RATE}
`N_ELEMENTS` (n) = {int(N_ELEMENTS):,}
`MAX_SCALEDOWN` = {MAX_SCALEDOWN} (recommended standard value)

Recommended parameters are: 

`size` (mκ) = {int(N_ELEMENTS * m_over_n * kappa):,}
`n_hashes` (k) = {n_hashes}
`marker_width` (ν) = {nu}
`n_marker_bits` (κ) = {kappa}
`secondary_scaledown` (uncorrected Array_0 β) = {np.ceil(uncorrected_beta * 1000)/1000:.3f}
`max_scaledown` (-) = {MAX_SCALEDOWN} (recommended standard value)
`n_secondaries` (number of Array_x's) = {n_secondaries}

Summary statistics:

* {np.sum(array_sizes, dtype=int):,} total bits ({np.sum(array_sizes) / (8 * 1024**2):.2f} Mb, {np.sum(array_sizes) / (8 * 1024**3):.2f} Gb)
* {np.sum(array_sizes) / N_ELEMENTS:.2f} bits per element
* {np.sum(array_sizes) / (N_ELEMENTS * 8):.2f} bytes per element
* Expected false positive rate (𝛼): {alpha:.4f}
* Expected error rate per bit in the primary array (p): {p:.4f}
""")